#Setup

In [1]:
! pip install pkbar

In [2]:
from google.colab import drive
from google.colab import files
import sys
import time

drive.mount('/content/gdrive/', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/MS Thesis/PSGD Paper/'
dpsgd_dir = base_dir + 'DPSGD/'
results_dir = base_dir + 'FNN/results_MNIST/'
logs_dir = base_dir + 'log'
sys.path.append(base_dir)
import preconditioned_stochastic_gradient_descent as psgd 

Mounted at /content/gdrive/


In [3]:
import matplotlib.pyplot as plt
import torch
from torch.autograd import grad
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import plotly.graph_objects as go
# from hessian_torch import Hessian
import numpy as np
import time
import math

import pkbar
from tabulate import tabulate
import scipy.io
from sklearn import metrics
import plotly.express as px
from torchsummary import summary


# Plot Functions

In [4]:
def check_model(data):
    image ,_ = data 
    a = myModel(image.to(device).view(-1,784))
    plt.figure(1)
    plt.imshow(image[0].view(28,28), cmap = 'gray')
    plt.figure(2)
    plt.imshow(a.view(28,28).cpu().detach().numpy(), cmap = 'gray')

def plot_loss_metrics(xaxis,yaxis,title, x_label,y_label):
 
  fig = go.Figure()
  i = 0
  if(xaxis != None):
    for opt in opts:
      fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1
  else:
    for opt in opts:
      fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1

  fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis_type="log")
  fig.show()
  fig.write_html(results_dir + title + ".html")



In [5]:
np.random.seed(0)

# Parameter Settings
BATCH_SIZE = 64
test_BATCH_SIZE = 1000
EPOCHS = 20
GAP = 100

# Data Download

In [6]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=True, download=True,           
                       transform=transforms.Compose([                       
                               transforms.ToTensor(), 
                              #  transforms.Normalize((0.5,), (0.5,))
                               ])),    
                        batch_size=BATCH_SIZE, shuffle=True, num_workers = 4, pin_memory = True)

test_loader = torch.utils.data.DataLoader(    
        datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(), 
                      #  transforms.Normalize((0.5,), (0.5,)) 
                       ])),    
                       batch_size=test_BATCH_SIZE, shuffle=True, num_workers=4, pin_memory = True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning:

The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [7]:
n_batches = np.ceil(len(train_loader.dataset)/BATCH_SIZE)
n_test_batches = np.ceil(len(test_loader.dataset)/test_BATCH_SIZE)

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
torch.cuda.device_count()
torch.cuda.get_device_name(0)

Running on the GPU


'Tesla K80'

# Model

In [19]:
"""input image size for the original myModel is 32x32, here is 28x28"""
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
dim = [784, 1000, 500, 250, 30, 250, 500, 1000, 784]

def initialize_weights():
    W1 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[0]+1,  dim[1])).clone().detach().requires_grad_(True)).to(device)
    W2 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[1]+1,  dim[2])).clone().detach().requires_grad_(True)).to(device)
    W3 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[2]+1,  dim[3])).clone().detach().requires_grad_(True)).to(device)
    W4 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[3]+1,  dim[4])).clone().detach().requires_grad_(True)).to(device)
    W5 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[4]+1,  dim[5])).clone().detach().requires_grad_(True)).to(device)
    W6 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[5]+1,  dim[6])).clone().detach().requires_grad_(True)).to(device)
    W7 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[6]+1,  dim[7])).clone().detach().requires_grad_(True)).to(device)
    W8 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[7]+1,  dim[8])).clone().detach().requires_grad_(True)).to(device)
    Ws = [W1, W2, W3, W4, W5, W6, W7, W8]
    return Ws

def myModel(x, return_all = False): 
    W1, W2, W3, W4, W5, W6, W7, W8 = Ws
   
    x1 = F.relu(x.mm(W1[:-1]) + W1[-1])
    x2 = F.relu(x1.mm(W2[:-1]) + W2[-1])
    x3 = F.relu(x2.mm(W3[:-1]) + W3[-1])
    x4 = F.relu(x3.mm(W4[:-1]) + W4[-1])
    x5 = F.relu(x4.mm(W5[:-1]) + W5[-1])
    x6 = F.relu(x5.mm(W6[:-1]) + W6[-1])
    x7 = F.relu(x6.mm(W7[:-1]) + W7[-1])
    y  = (x7.mm(W8[:-1]) + W8[-1])
    
    if return_all:
      return y, [x, x1, x2, x3, x4, x5, x6, x7, y]
    return y


In [15]:
"""input image size for the original myModel is 32x32, here is 28x28"""
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
dim = [784, 300, 100, 10, 100, 300, 784]

def initialize_weights():
    W1 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[0]+1,  dim[1])).clone().detach().requires_grad_(True)).to(device)
    W2 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[1]+1,  dim[2])).clone().detach().requires_grad_(True)).to(device)
    W3 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[2]+1,  dim[3])).clone().detach().requires_grad_(True)).to(device)
    W4 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[3]+1,  dim[4])).clone().detach().requires_grad_(True)).to(device)
    W5 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[4]+1,  dim[5])).clone().detach().requires_grad_(True)).to(device)
    W6 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[5]+1,  dim[6])).clone().detach().requires_grad_(True)).to(device)
   
    Ws = [W1, W2, W3, W4, W5, W6]
    return Ws

def myModel(x, return_all = False): 
    W1, W2, W3, W4, W5, W6 = Ws
   
    x = F.relu(x.mm(W1[:-1]) + W1[-1])
    x = F.relu(x.mm(W2[:-1]) + W2[-1])
    x = F.relu(x.mm(W3[:-1]) + W3[-1])
    x = F.relu(x.mm(W4[:-1]) + W4[-1])
    x = F.relu(x.mm(W5[:-1]) + W5[-1])
    y  = (x.mm(W6[:-1]) + W6[-1])
    
    return y


# Loss Function

In [10]:
def train_loss(data, target):
    y = myModel(data)
    loss = F.mse_loss(y, target)
    return loss

def test_loss():
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device).view(-1, 28*28)
            y = myModel(data)
            loss += F.mse_loss(y, data)
            
    return loss.item()/n_test_batches

def save_start_condition(trainlosslist, testlosslist, timelist):
    trainloss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
      data = data.to(device).view(-1, 28*28)
      loss= train_loss(data, data)
      trainloss += loss
      
    timelist.append(0)

    testloss = test_loss()

    trainlosslist.append(trainloss.item()/n_batches)
    testlosslist.append(testloss)
    print('Epoch: {}; train loss: {}; test loss: {},  time: {}'\
    .format(0, trainlosslist[-1], testlosslist[-1], np.sum(timelist)))

## SGD

In [26]:
torch.manual_seed(1)
Ws = initialize_weights()
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0

    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws)
        trainloss += loss
        
        with torch.no_grad():
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*grads[i]

        kbar.update(n, values=[("loss", loss.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()
    TestLoss.append(testloss)
    
    # step_size = 0.01**(1/9)*step_size
    kbar.add(1, values=[("val_loss", testloss)])

scipy.io.savemat(results_dir + 'sgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.11546933269704074; test loss: 0.11740000247955322,  time: 0
Epoch: 1/20
938/938 [==============================] - 12s 13ms/step - loss: 0.0754 - val_loss: 0.0636
Epoch: 2/20
938/938 [==============================] - 12s 13ms/step - loss: 0.0630 - val_loss: 0.0623
Epoch: 3/20
938/938 [==============================] - 12s 13ms/step - loss: 0.0611 - val_loss: 0.0592
Epoch: 4/20
938/938 [==============================] - 12s 13ms/step - loss: 0.0566 - val_loss: 0.0536
Epoch: 5/20
265/938 [=======>......................] - ETA: 8s - loss: 0.0530

KeyboardInterrupt: ignored

## Adam

In [27]:
torch.manual_seed(1)
Ws = initialize_weights()
m0 = [torch.zeros(W.shape).to(device) for W in Ws]
v0 = [torch.zeros(W.shape).to(device) for W in Ws]
step_size = 0.001
cnt = 0
n = 0
TrainLoss, TestLoss = [], []

times = []
with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws)#, create_graph=True)
        trainloss += loss

        with torch.no_grad():
            lmbd = min(cnt/(cnt+1), 0.9)
            m0 = [lmbd*old + (1.0-lmbd)*new for (old, new) in zip(m0, grads)]
            lmbd = min(cnt/(cnt+1), 0.999)
            v0 = [lmbd*old + (1.0-lmbd)*new*new for (old, new) in zip(v0, grads)]
            for i in range(len(Ws)):
                Ws[i] -= step_size*(m0[i]/torch.sqrt(v0[i] + 1e-8))
            cnt = cnt + 1
        kbar.update(n, values=[("loss", loss.item())])
        n += 1
        
    t1 = time.time() - t0
    times.append(t1)

    testloss = test_loss()

    TrainLoss.append(trainloss.item()/n_batches)
    
    TestLoss.append(testloss)
    
    # step_size = 0.01**(1/9)*step_size
    kbar.add(1, values=[("val_loss", testloss)])

scipy.io.savemat(results_dir + 'adam.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.11546933269704074; test loss: 0.11740000247955322,  time: 0
Epoch: 1/20
938/938 [==============================] - 13s 14ms/step - loss: 0.0340 - val_loss: 0.0233
Epoch: 2/20
938/938 [==============================] - 13s 14ms/step - loss: 0.0215 - val_loss: 0.0193
Epoch: 3/20
938/938 [==============================] - 13s 14ms/step - loss: 0.0183 - val_loss: 0.0172
Epoch: 4/20
938/938 [==============================] - 13s 14ms/step - loss: 0.0166 - val_loss: 0.0158
Epoch: 5/20
152/938 [===>..........................] - ETA: 11s - loss: 0.0158

KeyboardInterrupt: ignored

## Full Kronecker

In [34]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []
with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)

    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws, create_graph=True)
        trainloss += loss
        if n % 5 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
          Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]  
            
        with torch.no_grad():
            
            pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
        kbar.update(n, values=[("loss", loss.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()
    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    
    # step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, time: {}'\
    #  .format(epoch, TrainLoss[-1], TestLoss[-1], np.sum(times)))

# scipy.io.savemat(results_dir + 'Kron.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.11546933269704074; test loss: 0.11740000247955322,  time: 0
Epoch: 1/20
938/938 [==============================] - 35s 38ms/step - loss: 0.0566 - val_loss: 0.0234
Epoch: 2/20
938/938 [==============================] - 35s 38ms/step - loss: 0.0178 - val_loss: 0.0141
Epoch: 3/20
938/938 [==============================] - 35s 38ms/step - loss: 0.0127 - val_loss: 0.0115
Epoch: 4/20
 30/938 [..............................] - ETA: 43s - loss: 0.0114   

KeyboardInterrupt: ignored

## DAMPED KRONECKER

In [20]:
_tiny = 1.2e-38 
 # pi = (torch.trace(Ql)*Qr.shape[0])/(torch.trace(Qr)*Ql.shape[0])
    # 

    
def precond_grad_kron(Ql, Qr, Grad):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd**0.5))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd**0.5))*IR)

    return P1.mm(Grad).mm(P2)

def precond_grad_scaw(Ql, qr, Grad):
    P1 = Ql.t().mm(Ql)
    P2 = qr*qr
    pi = (torch.trace(P1)*P2.shape[0])/(torch.sum(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd**0.5))*IL)
    P2 = P2 + torch.sqrt((1/pi)*(eta + lambd**0.5)*IR)
    return P1.mm(Grad*(P2))


def update_lambda(loss1, loss2, M, lambd, omega):
    
    r = abs(loss2 - loss1)/(M)
    if r > 3/4:
      lambd = lambd*omega
    elif r < 1/4:
      lambd = lambd / omega
    return lambd


In [35]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5

step_size = 0.1
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after
eta = 1e-5

for epoch in range(EPOCHS):
    
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches
                      , epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        
        if n % 5 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
          Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]

        with torch.no_grad():     
            
            pre_grads = [precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % update_after == 0 and lambd > 1e-5:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.mse_loss(myModel(data), data)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)
        kbar.update(n, values=[("loss", loss.item())])
        n = n+1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    
    # step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, time: {}' .format(epoch+1, TrainLoss[-1], TestLoss[-1],np.sum(times)))

# scipy.io.savemat(results_dir + 'Kron_damped.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.11546933269704074; test loss: 0.11740000247955322,  time: 0
Epoch: 1/20
938/938 [==============================] - 41s 44ms/step - loss: 0.0532 - val_loss: 0.0234
Epoch: 2/20
938/938 [==============================] - 41s 44ms/step - loss: 0.0179 - val_loss: 0.0136
Epoch: 3/20
575/938 [=================>............] - ETA: 15s - loss: 0.0132

KeyboardInterrupt: ignored

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
dQs = [[torch.ones(W.shape[0],1).to(device), torch.ones(1,W.shape[1]).to(device)] for W in Ws]
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5

step_size = 0.1
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

lambd = 1
update_after = 5
omega = (19/20)**update_afte5
eta = 1e-5

for epoch in range(EPOCHS):
    
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches
                      , epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        
        if n % 5 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
          Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]

        with torch.no_grad():     
            
            pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            damp_grads = [((lambd+eta)**0.5)*g for g in grads]
            pre_grads = [pg+dg for (pg, dg) in zip(pre_grads, damp_grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % update_after == 0 and lambd > 1e-5:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.mse_loss(myModel(data), data)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)
        kbar.update(n, values=[("loss", loss.item())])
        n = n+1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    
    # step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, time: {}' .format(epoch+1, TrainLoss[-1], TestLoss[-1],np.sum(times)))

# scipy.io.savemat(results_dir + 'Kron_damped.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

In [33]:
lambd

2.0753033477680558e-07

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5

step_size = 0.1
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after
eta = 1e-5

for epoch in range(EPOCHS):
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches
                      , epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        
        if n % 10 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)

        with torch.no_grad():     
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % update_after == 0 and lambd > 1e-10:
                # M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                M = 0.5*min([torch.sum(g*pg) for (g, pg) in zip(grads, pre_grads)])
               
                loss2 = F.mse_loss(myModel(data), data)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)
        kbar.update(n, values=[("loss", loss.item())])
        n = n+1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    
    # step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, time: {}' .format(epoch+1, TrainLoss[-1], TestLoss[-1],np.sum(times)))

# scipy.io.savemat(results_dir + 'Kron_damped.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

## MIXED

In [42]:
def precond_kron(Ql, Qr, Pl, Pr, beta1, beta2):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd**0.5))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd**0.5))*IR)

    Pl = beta1*Pl + beta2*P1 
    Pr = beta1*Pr + beta2*P2 

    return [P1, P2, Pl, Pr]

def precond_kron2(Ql, Qr, Pl, Pr, beta1, beta2):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    Pl = beta1*Pl + beta2*P1 
    Pr = beta1*Pr + beta2*P2 
    return [P1, P2, Pl, Pr]

def precond_grad_kron2(Pl, Pr, Grad):
    return Pl.mm(Grad).mm(Pr)

torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
Ps = [[torch.zeros(W.shape[0]).to(device), torch.zeros(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, times)

T1 = 5
lambd = 1
update_after = 5
omega = (19/20)**update_after
eta = 1e-5

beta1 = 0.9
beta2 = 0.1

for epoch in range(EPOCHS):
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        
        if n % T1 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
          Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]

        with torch.no_grad():     
            
            Ps = [precond_kron(q[0], q[1], p[0], p[1], beta1, beta2) for (q, p) in zip(Qs, Ps)]
            pre_grads = [precond_grad_kron2(p[2], p[3], g) for (p, g) in zip(Ps, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % update_after == 0 and lambd > 1e-5:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.mse_loss(myModel(data), data)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)
        kbar.update(n, values=[("loss", loss.item())])
        n = n+1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    
scipy.io.savemat(results_dir + 'mod_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.11546933269704074; test loss: 0.11740000247955322,  time: 0
Epoch: 1/20
924/938 [============================>.] - ETA: 0s - loss: 0.0537

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1707399560>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1707399560>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

938/938 [==============================] - 44s 47ms/step - loss: 0.0533 - val_loss: 0.0234
Epoch: 2/20
935/938 [============================>.] - ETA: 0s - loss: 0.0178

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1707399560>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1707399560>
    self._shutdown_workers()
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

938/938 [==============================] - 44s 47ms/step - loss: 0.0178 - val_loss: 0.0142
Epoch: 3/20
490/938 [==============>...............] - ETA: 20s - loss: 0.0134

KeyboardInterrupt: ignored

In [46]:
def precond_kron(Ql, Qr, Pl, Pr, beta1, beta2):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd**0.5))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd**0.5))*IR)

    Pl = beta1*Pl + beta2*P1 
    Pr = beta1*Pr + beta2*P2 

    return [P1, P2, Pl, Pr]

def precond_kron2(Ql, Qr, Pl, Pr, beta1, beta2):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    Pl = beta1*Pl + beta2*P1 
    Pr = beta1*Pr + beta2*P2 
    return [P1, P2, Pl, Pr]

def precond_grad_kron2(Pl, Pr, Grad):
    return Pl.mm(Grad).mm(Pr)

torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
Ps = [[torch.zeros(W.shape[0]).to(device), torch.zeros(W.shape[1]).to(device), 
       torch.zeros(W.shape[0]).to(device), torch.zeros(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, times)

T1 = 1
lambd = 1
update_after = 5
omega = (19/20)**update_after
eta = 1e-5

beta1 = 0.9
beta2 = 0.1

for epoch in range(EPOCHS):
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        
        if n % T1 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
          Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]

        with torch.no_grad():     
            
            Ps = [precond_kron(q[0], q[1], p[2], p[3], beta1, beta2) for (q, p) in zip(Qs, Ps)]
            pre_grads = [precond_grad_kron2(p[2], p[3], g) for (p, g) in zip(Ps, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % update_after == 0 and lambd > 1e-5:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.mse_loss(myModel(data), data)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)
        kbar.update(n, values=[("loss", loss.item())])
        n = n+1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    
scipy.io.savemat(results_dir + 'mod_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.11546933269704074; test loss: 0.11740000247955322,  time: 0
Epoch: 1/20
938/938 [==============================] - 44s 46ms/step - loss: 0.0545 - val_loss: 0.0235
Epoch: 2/20
859/938 [==========================>...] - ETA: 3s - loss: 0.0181

KeyboardInterrupt: ignored

##KFAC

In [ ]:
from kfac import KFAC
import torch.nn as nn
import torch.optim as optim

class myModel_K(nn.Module):
    def __init__(self):
        super(myModel_K, self).__init__()

        self.fc1 = nn.Linear(dim[0], dim[1])
        self.fc2 = nn.Linear(dim[1], dim[2])
        self.fc3 = nn.Linear(dim[2], dim[3])
        self.fc4 = nn.Linear(dim[3], dim[4])
        self.fc5 = nn.Linear(dim[4], dim[5])
        self.fc6 = nn.Linear(dim[5], dim[6])
        self.fc7 = nn.Linear(dim[6], dim[7])
        self.fc8 = nn.Linear(dim[7], dim[8])


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = self.fc8(x)
        
        return x

def test_loss_K(model):
    model.eval()
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device).view(-1, 28*28*1)
            # data = data.view(-1, 28*28*1)
            y = model(data)
            loss += F.mse_loss(y, data)

    return loss.item()/n_test_batches

In [ ]:
torch.manual_seed(1)  
Ws = initialize_weights()
model = myModel_K()
model.to(device)
preconditioner = KFAC(model, 0.001, alpha=0.05)
lr0 = 0.1
optimizer = optim.SGD(model.parameters(), lr=lr0)
TrainLoss, TestLoss = [], []

times = []
with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    model.train()
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device).view(-1, 28*28*1)
        # data = data.view(-1, 28*28*1)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.mse_loss(output, data)

        trainloss += loss
        
        loss.backward()
        preconditioner.step()
        optimizer.step()

        kbar.update(n, values=[("loss", loss.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)

    TrainLoss.append(trainloss.item()/n_batches)
    
    # lr0 = 0.01**(1/9)*lr0
    optimizer.param_groups[0]['lr'] = lr0
    testloss = test_loss_K(model)
    kbar.add(1, values=[("val_loss", testloss)])

    TestLoss.append(testloss)
    
scipy.io.savemat(results_dir + 'KFAC.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

Epoch: 0; train loss: 0.11547256990282266; test loss: 0.11740000247955322,  time: 0
Epoch: 1/20
938/938 [==============================] - 68s 73ms/step - loss: 0.0342 - val_loss: 0.0178
Epoch: 2/20
938/938 [==============================] - 68s 72ms/step - loss: 0.0146 - val_loss: 0.0126
Epoch: 3/20
938/938 [==============================] - 68s 73ms/step - loss: 0.0114 - val_loss: 0.0107
Epoch: 4/20
938/938 [==============================] - 69s 73ms/step - loss: 0.0100 - val_loss: 0.0098
Epoch: 5/20
938/938 [==============================] - 69s 73ms/step - loss: 0.0090 - val_loss: 0.0090
Epoch: 6/20
938/938 [==============================] - 69s 73ms/step - loss: 0.0084 - val_loss: 0.0085
Epoch: 7/20
938/938 [==============================] - 68s 73ms/step - loss: 0.0079 - val_loss: 0.0081
Epoch: 8/20
938/938 [==============================] - 69s 73ms/step - loss: 0.0075 - val_loss: 0.0078
Epoch: 9/20
938/938 [==============================] - 69s 73ms/step - loss: 0.0072 - val_lo

## Shampoo

In [ ]:
limit = 250
def matrix_power(matrix, power):
    # use CPU for svd for speed up
    matrix = matrix.cpu()
    u, s, v = torch.svd(matrix)
    return (u @ s.pow_(power).diag() @ v.t()).cuda()

def initialize_shampoo(W, eps):
  if W.shape[0] >= limit:
    ql = eps*torch.ones(W.shape[0]).to(device)
  else:
    ql = eps*torch.eye(W.shape[0]).to(device)

  if W.shape[1] >= limit:
    qr = eps*torch.ones(W.shape[1]).to(device)
    
  else:
    qr = eps*torch.eye(W.shape[1]).to(device)
  return ql, qr

def update_shampoo(ql, qr, g):
  if ql.shape[0] >= limit:
    new_ql = ql + torch.diag(g.mm(g.t()))
    inv_ql = torch.pow(new_ql, -1/4)
  else:
    new_ql = ql + torch.diag(g.mm(g.t()))
    inv_ql = matrix_power(new_ql, -1/4)

  if qr.shape[0] >= limit:
    new_qr = qr + torch.diag(g.t().mm(g))
    inv_qr = torch.pow(new_qr, -1/4)
  else:
    new_qr = qr + g.t().mm(g)
    inv_qr = matrix_power(new_qr, -1/4)
  
  return new_ql, new_qr, inv_ql, inv_qr

def shampoo_grads(ql, qr, g):
  if ql.shape[0] >= limit and qr.shape[0] >= limit:
    pre_grads = ql.reshape(-1, 1) * g * qr.reshape(1, -1)
  elif ql.shape[0] >= limit and qr.shape[0] <= limit:
    pre_grads = ql.reshape(-1,1) * (g @ qr)
  elif ql.shape[0] <= limit and qr.shape[0] >= limit:
    pre_grads = (ql @ g) * qr.reshape(1,-1)
  else:
    pre_grads = ql @ g @ qr
  return pre_grads
  

torch.manual_seed(1)
Ws = initialize_weights()
Qs = [initialize_shampoo(W, 0.01) for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []

times = []

save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws)
        
        trainloss += loss
        
        
        with torch.no_grad():
            Qs = [update_shampoo(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            pre_grads = [shampoo_grads(q[2], q[3], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

        kbar.update(n, values=[("loss", loss.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    # step_size = 0.1**(1/9)*step_size

scipy.io.savemat(results_dir + 'shampoo.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

Epoch: 0; train loss: 0.11546933269704074; test loss: 0.11740000247955322,  time: 0
Epoch: 1/20
938/938 [==============================] - 11s 11ms/step - loss: 0.0526 - val_loss: 0.0384
Epoch: 2/20
938/938 [==============================] - 11s 12ms/step - loss: 0.0351 - val_loss: 0.0325
Epoch: 3/20
938/938 [==============================] - 11s 12ms/step - loss: 0.0299 - val_loss: 0.0288
Epoch: 4/20
938/938 [==============================] - 11s 11ms/step - loss: 0.0271 - val_loss: 0.0259
Epoch: 5/20
938/938 [==============================] - 11s 11ms/step - loss: 0.0254 - val_loss: 0.0249
Epoch: 6/20
938/938 [==============================] - 11s 12ms/step - loss: 0.0241 - val_loss: 0.0229
Epoch: 7/20
938/938 [==============================] - 11s 11ms/step - loss: 0.0231 - val_loss: 0.0220
Epoch: 8/20
938/938 [==============================] - 11s 12ms/step - loss: 0.0223 - val_loss: 0.0215
Epoch: 9/20
938/938 [==============================] - 11s 12ms/step - loss: 0.0217 - val_lo

# Comparison

In [ ]:
opts = ['sgd','adam','KFAC','shampoo','Kron', 'Kron_damped', 'mod_psgd', 'DPSGD-S']
total_train_time = {}
opts_data = {}
times = {}
train_times = {}
test_times = {}
train_losses = {}
test_losses = {}
train_accs = {}
test_accs = {}


for opt in opts:
	opts_data[opt] = scipy.io.loadmat(results_dir+opt+'.mat')	

In [ ]:
colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#000000', '#FF33E6','#888A0B','#8A0B1E','#B498DF','#1B786D']
# colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#FFF933','#000000','#33E0FF','#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']

In [ ]:
for opt in opts:
  # print(opt)
  data = opts_data[opt]
  times[opt] = data.get('Time')
  train_times[opt] = np.cumsum(times[opt])
  test_times[opt] = np.cumsum(times[opt])
  total_train_time[opt] = np.sum(times[opt])
  train_losses[opt] = data.get('TrainLoss').reshape(EPOCHS+1,)
  test_losses[opt] = data.get('TestLoss').reshape(EPOCHS+1,)


In [ ]:
# plot train_losses vs Iterations
plot_loss_metrics(None,train_losses,'Train Loss vs EPOCHS', 'EPOCHS','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(None,test_losses,'Test Loss vs EPOCHS', 'EPOCHS','Test Loss')
# # plot test_losses vs Iterations
plot_loss_metrics(train_times,train_losses,'Train Loss vs Time', 'Time','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(test_times,test_losses,'Test Loss vs Time', 'Time','Test Loss')